In [14]:
import pandas as pd
import os
import re

In [28]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth

In [ ]:
os.environ["SPOTIPY_CLIENT_ID"]=""
os.environ["SPOTIPY_CLIENT_SECRET"]=""
os.environ["SPOTIPY_REDIRECT_URI"]="https://example.org/callback"
scope="user-library-read"
sp=spotipy.Spotify(auth_manager=SpotifyOAuth(scope=scope))

In [35]:
artist_urn="spotify:artist:3meJIgRw7YleJrmbpbJK6S"
artist=sp.artist(artist_urn)

In [59]:
albums_dicts = [sp.artist_albums(artist_urn, limit=50)]
total_albums = albums_dicts[0]["total"]
for offset in range(50,total_albums,50):
    albums_dicts.append(sp.artist_albums(artist_urn, limit=50, offset=offset))

In [20]:
folgen_pattern = re.compile(r"Folge (?P<number>[0-9]+):\s+(?P<title>.*)")

In [91]:
folgen={}
for albums in albums_dicts:
    number,title = "",""
    match=None
    for album in albums["items"]:
        match = re.match(
            r"Folge (?P<number>[0-9]+):\s+(?P<title>.*)",
            album["name"]
        )
        if match is None:
            match=re.match(
                r"(?P<number>[0-9]+)/(?P<title>.*)",
                album['name']
            )
        if match:
            number=match.groupdict().get("number")
            title=match.groupdict().get("title")
            folgen[int(number)]= {"title":title, "number":int(number) } | {k:v for k,v in album.items() if k not in ["available_markets",]}


In [92]:
import json
json.dump(folgen, open("dreifragezeichen_folgen.json","w"))